
## **ChipNet**

This notebook demonstrates the implementation of the paper - **ChipNet: Budget-Aware Pruning with Heaviside Continuous Approximations**

Steps to train a baseline model and then compress it for a given budget are as follows: 
*   Load the YAML file.
*   Load dataset and create dataloaders.
*   Create ChipNet object and pass the parameters in the form of a dictionary.
*   Pass the dataloaders into the compress_model method to obtain the compressed model.

Since this is a demo notebook the number of epochs have been set to 1, 1 and 2 respectively for pretraining, pruning and finetuning respectively.



In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import sys
sys.path.append("../../../")

import torch
import matplotlib.pyplot as plt
from torchvision import transforms
from trailmet.models import ModelsFactory
from trailmet.datasets.classification import DatasetFactory
import yaml

/opt/conda/envs/py117/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Loading the yaml file for the configurations.

In [2]:
import os
with open(os.path.join("chipnet.yaml"), 'r') as stream:
    data_loaded = yaml.safe_load(stream)
print(data_loaded)

{'CHIPNET_ARGS': {'BUDGET_TYPE': 'channel_ratio', 'TARGET_BUDGET': 0.5, 'BUDGET_LOSS_WEIGHTAGE': 30, 'CRISPNESS_LOSS_WEIGHTAGE': 10, 'BETA_INCREMENT': 5, 'GAMMA_INCREMENT': 2, 'STEEPNESS': 100}, 'PRETRAIN': {'EPOCHS': 1, 'TEST_ONLY': False, 'OPTIMIZER': 'SGD', 'LR': 0.05, 'SCHEDULER_TYPE': 1, 'WEIGHT_DECAY': 0.001}, 'PRUNE': {'EPOCHS': 1, 'TEST_ONLY': False, 'OPTIMIZER': 'SGD', 'LR': 0.05, 'WEIGHT_DECAY': 0.001}, 'FINETUNE': {'EPOCHS': 2, 'TEST_ONLY': False, 'OPTIMIZER': 'SGD', 'LR': 0.05, 'SCHEDULER_TYPE': 1, 'WEIGHT_DECAY': 0.001}, 'wandb': True, 'insize': 32}


### Loading the model.

In [3]:
model = ModelsFactory.create_model(name='resnet50', num_classes=100, pretrained=False, **data_loaded)

### Loading the dataset.

In [4]:
train_transform = transforms.Compose(
[transforms.ToTensor()])

val_transform = transforms.Compose(
[transforms.ToTensor()])

test_transform = transforms.Compose(
[transforms.ToTensor()])

transforms1 = {
    'train': train_transform, 
    'val': val_transform, 
    'test': test_transform}
def train_target_transform(label):
    return label

def val_target_transform(label):
    return label

def test_target_transform(label):
    return label

target_transforms = {
    'train': None, 
    'val': None, 
    'test': None}


cifar_dataset = DatasetFactory.create_dataset(name = 'CIFAR100', 
                                        root = "./data_dir/",
                                        split_types = ['train', 'val', 'test'],
                                        val_fraction = 0.2,
                                        transform = transforms1,
                                        target_transform = target_transforms
)


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


### Creating the dataloaders.

In [5]:
train_loader = torch.utils.data.DataLoader(
        cifar_dataset['train'], batch_size=64, 
        sampler=cifar_dataset['train_sampler'],
        num_workers=0
    )
val_loader = torch.utils.data.DataLoader(
        cifar_dataset['val'], batch_size=64, 
        sampler=cifar_dataset['val_sampler'],
        num_workers=0
    )
test_loader = torch.utils.data.DataLoader(
        cifar_dataset['test'], batch_size=64, 
        sampler=cifar_dataset['test_sampler'],
        num_workers=0
    )

### Creating the method's object followed by compression

In [6]:
from trailmet.algorithms.prune.chipnet import ChipNet

In [7]:
a = ChipNet(model, {'train': train_loader, 'val': val_loader, 'test': test_loader}, **data_loaded)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: animesh-007. Use `wandb login --relogin` to force relogin


In [8]:
a.compress_model()

Pretrainning the model


Training PRETRAIN Epoch [0] (625 / 625 Steps) (batch time=0.04972s) (data time=0.01012s) (loss=3.91660): 100%|| 625/625 [00:33<00:00, 18.79it/s]
Validating PRETRAIN Epoch [0] (157 / 157 Steps) (batch time=0.02602s) (loss=3.73506) (top1=18.75000) (top5=43.75000): 100%|| 157/157 [00:03<00:00, 43.23it/s]


 * acc@1 9.860 acc@5 30.950


Validating PRETRAIN Epoch [0] (157 / 157 Steps) (batch time=0.01580s) (loss=3.99817) (top1=6.25000) (top5=25.00000): 100%|| 157/157 [00:03<00:00, 44.03it/s] 


 * acc@1 10.090 acc@5 31.260
Test Accuracy: 10.09000015258789 | Valid Accuracy: 9.859999656677246
preparing model for pruning
Starting epoch 0 / 1


...

 * acc@1 4.450 acc@5 17.650


Training FINETUNE Epoch [1] (625 / 625 Steps) (batch time=0.05819s) (data time=0.00911s) (loss=4.11850): 100%|| 625/625 [00:38<00:00, 16.42it/s]
Validating FINETUNE Epoch [1] (157 / 157 Steps) (batch time=0.02904s) (loss=4.34621) (top1=0.00000) (top5=18.75000): 100%|| 157/157 [00:04<00:00, 32.64it/s]


 * acc@1 4.210 acc@5 18.630


Validating FINETUNE Epoch [1] (157 / 157 Steps) (batch time=0.02115s) (loss=4.69811) (top1=0.00000) (top5=31.25000): 100%|| 157/157 [00:04<00:00, 35.35it/s]

 * acc@1 4.530 acc@5 18.000
Test Accuracy: 4.529999732971191 | Valid Accuracy: 4.449999809265137
